# Library

In [1]:
from typing import List, Dict, Any, Tuple, Callable, Union, Optional

import sys, os, cv2, re
import torch
from torch import nn
import numpy as np
import pandas as pd

from scripts.default_setting import *

from GGUtils.utils.path import do_or_load, GetAbsolutePath
from GGUtils.img.viewer import show_img, show_imgs
from GGUtils.utils.utils import time_checker
from GGDL.utils import set_seed_everything, make_basic_directory, tensor_to_img, GetDevice, Option
from GGDL.idx_dict.key_df import make_basic_key_df, binary_label_convertor
from GGDL.idx_dict.make_dict import make_stratified_idx_dict
from GGDL.data_loader.dataset import ImgDataset, GetLoader, show_dataset_img
from GGDL.model.vision import Classification, TimmHelper
from GGDL.model.fine_tuning import Tuner
from GGDL.model.optimzer import Optim, LabelDtype
from GGDL.pipeline.pipeline import BackPropagation

from GGImgMorph.scenario import sample_augment      # 증강 알고리즘

# Option

In [2]:
# process id
PROCESS_ID = os.getpid()
print("해당 process의 id: ", PROCESS_ID)

# 학습 상태 출력
VERBOSE = True

해당 process의 id:  3813287


### Option 1. model

In [3]:
# timm에서 사용하고자 하는 모델의 이름을 찾는다.
_model_name_ptn = "mobilenetv1_.+_r224"
TimmHelper.search(_model_name_ptn)

['mobilenetv1_100.ra4_e3600_r224_in1k',
 'mobilenetv1_100h.ra4_e3600_r224_in1k',
 'mobilenetv1_125.ra4_e3600_r224_in1k']

In [4]:
# model 관련 설정
MODEL_NAME = 'mobilenetv1_100.ra4_e3600_r224_in1k'      # baseline model
IMG_CHANNEL = 3              # image의 channel 크기
CLASS_SIZE = 0               # model이 추론할 class의 크기
PRETRAINED = True
USE_AMP = True               # AMP 사용 여부
USE_CLIPPING = True          # Grad clipping 사용 여부


# Custom header
class HeaderBlock(nn.Module):
    def __init__(self, input_dim:int, output_dim:int, dropout_prob:float):
        super(HeaderBlock, self).__init__()
        self.batch_norm = nn.BatchNorm1d(input_dim)
        self.linear = nn.Linear(input_dim, output_dim)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        x = self.batch_norm(x)
        x = self.linear(x)
        x = self.gelu(x)
        return self.dropout(x)


def custom_header(x:int)->nn.Module:
    """
    Pre-Activation Batch Normalization
    - 깊은 Backbone 모델의 header이므로, Internal Covariate Shift 문제 해결을 위해 사용

    BottleNeck
    - 정보를 확장하여 중요한 정보만 남겨, 계산 효율성을 유지하면서 높은 표현력 제공
    """
    header = nn.Sequential(
        HeaderBlock(input_dim=x, output_dim=1024, dropout_prob=0.1),
        HeaderBlock(input_dim=1024, output_dim=512, dropout_prob=0.3),
        HeaderBlock(input_dim=512, output_dim=128, dropout_prob=0.5),
        nn.Linear(128, 1)
    )
    return header

CUSTOM_HEAD_FN = custom_header

### Option 2. pipe line setting

In [5]:
# gpu 상태 확인
GET_DEVICE = GetDevice()
GET_DEVICE.summary()

CUDA is available.
GPU size: 2
------------------------------------------------------------
GPU number: 0
Name: NVIDIA GeForce RTX 3080 Ti
Computer capability: 8.6
VRAM: 12GB
------------------------------------------------------------
GPU number: 1
Name: NVIDIA GeForce GTX 750
Computer capability: 5.0
VRAM: 1GB
------------------------------------------------------------


In [6]:
# device 설정
GPU = 0
DEVICE = GET_DEVICE(GPU)
torch.cuda.set_device(DEVICE)

# data loader 관련 설정
DATASET_CLASS = ImgDataset          # dataset의 class
BATCH_SIZE = 16
IMG_SIZE = 224                      # 고정된 image의 크기
RESIZE_HOW = 2                      # resize 방법
RESIZE_HOW_LIST = [2, 3, 4]         # 무작위 resize 시, 방법의 list
RESIZE_PADDING_COLOR = "random"     # resize padding 시, pixel의 색
WORKER = 0                          # DataLoader의 num_worker

# early stopping 시 경로
ESTOP_PATH = f"{SOURCE}/{ESPOINT_DIR}/process_{PROCESS_ID}"

# process 진행 중 생성되는 파일들이 저장되는 초기 디렉터리 초기화 여부
MAKE_NEW_DEFAULT_DIR = True

### Option 3. Hyper Parameter

In [7]:
HP_DICT = {
    'lr':0.0001,
    'betas':(0.9, 0.999),
    'eps':1e-08,
    'clipping_max_norm':5
}
OPTIM_INS = Optim(name='Adam', hp_dict=HP_DICT)


LOSS_FN = nn.BCEWithLogitsLoss()
LABEL_TYPE_INS = LabelDtype(loss_fn=LOSS_FN)

# Process
### Process 1. make key_df
* key_df는 img의 절대 경로("path")와 label("label") 두 개의 컬럼으로 구성된 DataFrame 이다.

In [8]:
# 경로 정보
TRAIN_SET = "/mnt/d/rawdata/dogs-vs-cats/train/"        # train set의 경로
TEST_SET = "/mnt/d/rawdata/dogs-vs-cats/test1/"         # test set의 경로

In [9]:
# key_df 생성
path_list = GetAbsolutePath(None).get_all_path(parents_path=TRAIN_SET)
key_df = make_basic_key_df(
    paths=path_list,
    labels=[re.split(r".+/", i, maxsplit=1)[1].split('.')[0] for i in path_list]
)
# label을 이진 분류로 변환
key_df['label'] = binary_label_convertor(array=key_df['label'], positive_class='dog')

# 이해를 돕기 위한 key_df 출력
key_df

,path,label
0,/mnt/d/rawdata/dogs-vs-cats/train/cat.0.jpg,0
1,/mnt/d/rawdata/dogs-vs-cats/train/cat.1.jpg,0
2,/mnt/d/rawdata/dogs-vs-cats/train/cat.10.jpg,0
3,/mnt/d/rawdata/dogs-vs-cats/train/cat.100.jpg,0
4,/mnt/d/rawdata/dogs-vs-cats/train/cat.1000.jpg,0
...,...,...
24995,/mnt/d/rawdata/dogs-vs-cats/train/dog.9995.jpg,1
24996,/mnt/d/rawdata/dogs-vs-cats/train/dog.9996.jpg,1
24997,/mnt/d/rawdata/dogs-vs-cats/train/dog.9997.jpg,1
24998,/mnt/d/rawdata/dogs-vs-cats/train/dog.9998.jpg,1


### Process 2. make idx_dict

In [10]:
MAKE_NEW_IDX_DICT = True                            # idx_dict을 새로 생성할지 여부
IDX_DICT_PATH = f"{SOURCE}/idx_dict.pickle"         # idx_dict이 저장될 경로

# idx_dict 생성 방식
K_SIZE = 5                  # k-fold의 크기 (Stratified sampling)
TEST_RATIO = 0.2            # test dataset ratio
VALID_RATIO = 0.1           # validation dataset ratio, None인 경우 생성하지 않음

In [11]:
# 기초 디렉터리 생성
make_basic_directory(source=SOURCE, estop_dir=ESPOINT_DIR, log=LOG, result=RESULT, make_new=MAKE_NEW_DEFAULT_DIR)

# idx_dict 생성
IDX_DICT = do_or_load(
    savepath=IDX_DICT_PATH, makes_new=MAKE_NEW_IDX_DICT, 
    fn=make_stratified_idx_dict,
    key_df=key_df, stratified_columns=['label'], is_binary=True,
    path_col='path', label_col='label', 
    k_size=K_SIZE, test_ratio=TEST_RATIO, valid_ratio=VALID_RATIO
)

### Process 3. make option instance

In [12]:
option = Option(
    process_id=PROCESS_ID,
    model_name=MODEL_NAME, pretrained=PRETRAINED, device=DEVICE, idx_dict=IDX_DICT,
    optimizer=OPTIM_INS, loss_fn=LOSS_FN, label_type_fn=LABEL_TYPE_INS, use_amp=USE_AMP, use_clipping=USE_CLIPPING,
    img_size=IMG_SIZE, resize_how=RESIZE_HOW, resize_how_list=RESIZE_HOW_LIST, resize_padding_color=RESIZE_PADDING_COLOR,
    dataset_class=DATASET_CLASS, augments=sample_augment, batch_size=BATCH_SIZE, worker=WORKER,
    img_channel=IMG_CHANNEL, class_size=CLASS_SIZE, custom_header=CUSTOM_HEAD_FN,
    tuner_how=2, hp_dict=HP_DICT, verbose=VERBOSE,
    log_parents=f"{LOG}", results_parents=f"{RESULT}", espoint_parents=f"{SOURCE}/{ESPOINT_DIR}"
)

### Process 4. model training

In [13]:
from GGUtils.utils.path import new_dir_maker, make_null_list_pickle, load_pickle, save_pickle

In [14]:
class Log:
    def __init__(self, log_dir, process_id):

        self.parents_path = f"{log_dir}/{process_id}"
        self.iterator_py = f"{self.parents_path}/iterator.py"
        self.epoch_py = f"{self.parents_path}/epoch.py"
        self.k = None


    def make(self):
        new_dir_maker(self.parents_path)
        make_null_list_pickle(self.iterator_py)
        make_null_list_pickle(self.epoch_py)


    def read(self, open_epoch_py:bool=False)->List[Any]:
        pickle_path = self.epoch_py if open_epoch_py else self.iterator_py
        return load_pickle(pickle_path)
    

    def write(self, data:List[Any], open_epoch_py:bool=False):
        pickle_path = self.epoch_py if open_epoch_py else self.iterator_py
        save_pickle(data, pickle_path)

In [15]:
# 학습 전 모든 seed 고정
set_seed_everything(seed=SEED)

log_ins = Log(log_dir=option.log_parents, process_id=option.process_id)
log_ins.make()

for k in option.idx_dict.keys():
    
    k_idx_dict = option.idx_dict[k]     # k-fold에 대한 idx_dict
    break

In [16]:
# Log 설정
log_ins.k = k

# Data Loader 정의
loader = GetLoader(
    dataset_class=option.dataset_class, idx_dict=k_idx_dict,
    augments=option.augments, batch_size=option.batch_size, workers=option.worker, 
    resize=option.img_size, resize_how=option.resize_how, resize_how_list=option.resize_how_list,
    resize_padding_color = option.resize_padding_color
)

# model 정의
model = Classification(
    model_name=option.model_name, pretrained=option.pretrained, 
    channel=option.img_channel, class_size=option.class_size,
    custom_head_fn=option.custom_header
).to(option.device)

# Optimizer 설정
optimizer = option.optimizer(param=model.parameters())
grad_scaler = torch.GradScaler(device=option.device) if option.use_amp else None
back_propagation = BackPropagation(
    optimizer=optimizer, use_amp=option.use_amp, grad_scaler=grad_scaler,
    use_clipping=option.use_clipping, 
    max_norm=option.hp_dict['clipping_max_norm'] if 'clipping_max_norm' in option.hp_dict else None
)

# Fine tuning 방법 정의
tuner = Tuner(model, how=2, freezing_ratio=0.9)
tuner(epoch=0)      # model parameter 초기 변화

In [ ]:
import time

In [ ]:
class Classification:
    def __init__(self, model, loader, optimizer, back_propagation, log_ins, option):
        self.model = model
        self.loader = loader
        self.optimizer = optimizer
        self.back_propagation = back_propagation
        self.log_ins = log_ins
        self.option = option


    def _fit_epoch(self):
        pass


    def _fit_iterator(self)->Tuple[float, str]:
        start = time.time()
        loss_list = []
        for imgs, labels in loader.train:

            # load to device
            imgs = imgs.to(self.option.device)
            # label dtype을 loss_fn에 맞게 수정 및 shape 등 변형
            labels = option.label_type_fn(labels).reshape(-1, 1).to(self.option.device)

            # AMP
            with torch.autocast(device_type=option.device, enabled=self.option.use_amp):
                output = model(imgs)
                loss = option.loss_fn(output, labels)
                
            # back propagation
            back_propagation(loss)

            # loss log
            loss_list.append(loss.item())
        
        # iterator의 loss 평균, 종료 시간 출력
        return np.mean(loss_list), time_checker(start)
    



In [19]:
TEST_INS = Classification(
    model=model, loader=loader.train, optimizer=optimizer, 
    back_propagation=back_propagation, log_ins=log_ins, option=option
)

In [ ]:
start = time.time()
loss_list = []
for imgs, labels in loader.train:

    # load to device
    imgs = imgs.to(TEST_INS.option.device)
    # label dtype을 loss_fn에 맞게 수정 및 shape 등 변형
    labels = option.label_type_fn(labels).reshape(-1, 1).to(TEST_INS.option.device)

    # AMP
    with torch.autocast(device_type=option.device, enabled=TEST_INS.option.use_amp):
        output = model(imgs)
        loss = option.loss_fn(output, labels)
        
    # back propagation
    back_propagation(loss)

    # loss log
    loss_list.append(loss.item())

    break

In [20]:
# from torch.optim import lr_scheduler


# class Scheduler:
#     def __init__(self, name:str, hp_dict:Dict[str, Any]):
#         self.methods = {
#             'LambdaLR': lr_scheduler.LambdaLR,
#             'MultiplicativeLR': lr_scheduler.MultiplicativeLR,




#             'StepLR': lr_scheduler.StepLR,
#             'MultiStepLR': lr_scheduler.MultiStepLR,
#             'ExponentialLR': lr_scheduler.ExponentialLR,
#             'CosineAnnealingLR': lr_scheduler.CosineAnnealingLR,
#             'CyclicLR': lr_scheduler.CyclicLR,
#             'CosineAnnealingWarmRestarts': lr_scheduler.CosineAnnealingWarmRestarts,
#             'ReduceLROnPlateau': lr_scheduler.ReduceLROnPlateau,
#             'OneCycleLR': lr_scheduler.OneCycleLR,
#             'PolynomialLR': lr_scheduler.PolynomialLR,
#             'LinearLR': lr_scheduler.LinearLR,
#             'ConstantLR': lr_scheduler.ConstantLR,
            
            
            
            
            
#             'ChainedScheduler': lr_scheduler.ChainedScheduler,
#             'SequentialLR': lr_scheduler.SequentialLR,
            
#         }


#     def lambdalr(self):
#         pass


# class Schedulers:
#     def __init__(self):
#         pass